In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, LLM, Task, Crew, Process

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## Async Execution

In [2]:
from pydantic import BaseModel

class AIResearchFindings(BaseModel):
    """Represents structured research on AI breakthroughs."""
    title: str
    key_findings: list[str]

class AIRegulationFinding(BaseModel):
    """Represents structured research on AI regulations."""
    region: str
    key_policies: list[str]

class AIRegulationFindings(BaseModel):
    """Represents structured research on AI regulations."""
    key_regulations: list[AIRegulationFinding]

class FinalAIReport(BaseModel):
    """Combines AI research & regulation analysis into a report."""
    executive_summary: str
    key_trends: list[str]


In [3]:
from crewai import Agent

# Researcher for AI breakthroughs
research_agent = Agent(
    role="AI Researcher",
    goal="Find and summarize the latest AI breakthroughs",
    backstory="An expert AI researcher who tracks technological advancements.",
    llm=llm,
    verbose=True,
)

# Analyst for AI regulations
regulation_agent = Agent(
    role="AI Policy Analyst",
    goal="Analyze global AI regulations and summarize policies",
    backstory="A government policy expert specializing in AI ethics and laws.",
    llm=llm,
    verbose=True,
)

# Writer for the final AI report
writer_agent = Agent(
    role="AI Report Writer",
    goal="Write a structured report combining AI breakthroughs and regulations",
    backstory="A professional technical writer who crafts AI research reports.",
    llm=llm,
    verbose=True,
)


In [4]:
from crewai import Task

# Task 1: AI Breakthroughs Research (Asynchronous)
research_ai_task = Task(
    description="Research the latest AI advancements and summarize key breakthroughs.",
    expected_output="A structured list of AI breakthroughs.",
    agent=research_agent,
    output_pydantic=AIResearchFindings,
    async_execution=True  # Runs asynchronously
)

# Task 2: AI Regulation Analysis (Asynchronous)
research_regulation_task = Task(
    description="Analyze the latest AI regulations worldwide and summarize key policies.",
    expected_output="A structured list of AI regulations by region.",
    agent=regulation_agent,
    output_pydantic=AIRegulationFindings,
    async_execution=True  # Runs asynchronously
)

# Task 3: Generate AI Research Report (Depends on the first two tasks)
generate_report_task = Task(
    description="Write a structured report summarizing AI breakthroughs and regulations.",
    expected_output="A final AI report summarizing both aspects.",
    agent=writer_agent,
    output_pydantic=FinalAIReport,
    context=[research_ai_task, research_regulation_task]  # Waits for these tasks to complete
)


In [ ]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[research_agent, regulation_agent, writer_agent],
    tasks=[research_ai_task, research_regulation_task, generate_report_task],
    verbose=True
)

# Execute the workflow
result = ai_research_crew.kickoff()

# Print the final AI report
print("\n=== Generated AI Report ===")
print(result.raw)